In [3]:
import numpy as np
import matplotlib as plt
import nfl_data_py as nfl
import pandas as pd

In [2]:
year = 2020
nfl_2020 = nfl.import_pbp_data([year],  downcast=True, cache=False, alt_path=None)

2020 done.
Downcasting floats.


In [4]:
columns = nfl_2020.columns

In [30]:
[print(x) for x in columns if 'tac' in x]

solo_tackle
tackled_for_loss
assist_tackle
tackle_for_loss_1_player_id
tackle_for_loss_1_player_name
tackle_for_loss_2_player_id
tackle_for_loss_2_player_name
solo_tackle_1_team
solo_tackle_2_team
solo_tackle_1_player_id
solo_tackle_2_player_id
solo_tackle_1_player_name
solo_tackle_2_player_name
assist_tackle_1_player_id
assist_tackle_1_player_name
assist_tackle_1_team
assist_tackle_2_player_id
assist_tackle_2_player_name
assist_tackle_2_team
assist_tackle_3_player_id
assist_tackle_3_player_name
assist_tackle_3_team
assist_tackle_4_player_id
assist_tackle_4_player_name
assist_tackle_4_team
tackle_with_assist
tackle_with_assist_1_player_id
tackle_with_assist_1_player_name
tackle_with_assist_1_team
tackle_with_assist_2_player_id
tackle_with_assist_2_player_name
tackle_with_assist_2_team


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [7]:
team_ids = np.unique( nfl_2020['home_team'].to_list()  )

In [10]:
team_ids

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'LV',
       'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype='<U3')

In [15]:
assert(len(team_ids)) == 32, 'Post expansion'

In [27]:
seattle_home = nfl_2020[ nfl_2020['home_team'] == 'SEA' ]
seattle_away = nfl_2020[ nfl_2020['away_team'] == 'SEA' ]

we want the tackles done **by** seattle, so seattle can not be the team which possesses the ball

In [28]:
seattle_home = seattle_home[seattle_home['possession_team'] != 'SEA']
seattle_away = seattle_away[seattle_away['possession_team'] != 'SEA']

**home tackles**

In [34]:
solo = len(seattle_home[ seattle_home['solo_tackle'] ==1 ] )
assisted = len(seattle_home[ seattle_home['assist_tackle'] ==1 ] )
seattle_tackles_at_home = solo + assisted

In [35]:
seattle_tackles_at_home

486

**away tackles**

In [36]:
solo = len(seattle_away[ seattle_away['solo_tackle'] ==1 ] )
assisted = len(seattle_away[ seattle_away['assist_tackle'] ==1 ] )
seattle_tackles_away = solo + assisted

In [37]:
seattle_tackles_away

444

:-) less tackles away!!! okay generalize it

In [48]:
def filt_by_team(team, nfl_2020):
    #assert type(team) == str, 'something went wrong with team ' + str(team) + ' which is of team ' + str(type(team))
    home = nfl_2020[ nfl_2020['home_team'] == str(team) ]
    away = nfl_2020[ nfl_2020['away_team'] ==  str(team) ]
    
    home = home[home['possession_team'] != str(team)]
    away = away[away['possession_team'] != str(team)]
    
    return home, away

def total_tackles_home( home ):
    solo = len(home[ home['solo_tackle'] == 1 ] )
    assisted = len(home[ home['assist_tackle'] == 1 ] )
    tackles_at_home = solo + assisted
    return tackles_at_home
    
def total_tackles_away( away ):
    solo = len(away[ away['solo_tackle'] ==1 ] )
    assisted = len(away[away['assist_tackle'] ==1 ] )
    tackles_away = solo + assisted
    return tackles_away


Make dictionary for data where each key is a different team

In [43]:
data = {}
for team in team_ids:
    data[str(team)] = np.zeros((2))


In [49]:

for team in team_ids:
    home, away = filt_by_team(team, nfl_2020)
    
    home_tackles, away_tackles = total_tackles_home( home ), total_tackles_away( away )
    
    data[team][0], data[team][1] = home_tackles, away_tackles
    
    

In [50]:
data.keys()

dict_keys(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS'])

In [51]:
data['ARI']

array([427., 423.])